In [1]:
import plotly.graph_objs as go
import plotly.io as pio
import json
import re

def extract_json_section(input_string, tag):
    """Extract JSON section between specified XML-like tags."""
    pattern = f"<{tag}>(.*?)</{tag}>"
    match = re.search(pattern, input_string, re.DOTALL)
    if match:
        return match.group(1).strip()
    return None

def parse_json(json_str):
    """Attempt to parse JSON with relaxed rules."""
    try:
        return json.loads(json_str)
    except json.JSONDecodeError:
        # Try replacing single quotes with double quotes and parsing again
        try:
            fixed_json_str = json_str.replace("'", '"')
            return json.loads(fixed_json_str)
        except json.JSONDecodeError as e:
            raise ValueError(f"Invalid JSON: {e}")

def plot_from_ai_output_v2(input_string, img_path):
    # Extract JSON sections
    data_json = extract_json_section(input_string, "data")
    layout_json = extract_json_section(input_string, "layout")
    config_json = extract_json_section(input_string, "config")

    # Parse JSON strings with relaxed rules
    data = parse_json(data_json) if data_json else None
    layout = parse_json(layout_json) if layout_json else None
    config = parse_json(config_json) if config_json else None

    if not data or not layout:
        raise ValueError("Invalid or missing data or layout JSON.")

    # Prepare traces for the plot
    traces = []
    for trace_data in data:
        trace_type = trace_data.get('type')

        if trace_type == 'heatmap':
            trace = go.Heatmap(
                z=trace_data.get('z', []),
                x=trace_data.get('x', []),
                y=trace_data.get('y', []),
                name=trace_data.get('name', ''),
                colorscale=trace_data.get('colorscale', 'Viridis'),
                hoverinfo='x+y+z'
            )

            traces.append(trace)

    # Create figure with the extracted layout and data
    fig = go.Figure(data=traces, layout=layout)

    # Render the figure with the config
    pio.show(fig, config=config)

    # Save the figure as an image (optional)
    fig.write_image(img_path)


In [2]:
import google.generativeai as genai
from config import gemini_key

C:\Users\itsme\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
def input_image_setup(file_loc):
    from pathlib import Path

    if not (img := Path(file_loc)).exists():
        raise FileNotFoundError(f"Could not find image: {img}")

    image_parts = [
        {
            "mime_type": "image/jpeg",
            "data": Path(file_loc).read_bytes()
            }
        ]
    return image_parts
def get_image_info(image_loc, prompt):
    genai.configure(api_key=gemini_key)
    # Set up the model
    generation_config = {
        "temperature":0,
        "top_p":1,
        "top_k":32,
        "max_output_tokens":5000,
    }
    
    model = genai.GenerativeModel(model_name="gemini-1.5-pro", generation_config=generation_config)

    input_prompt = """ You are an expert in data visualization and graph analysis, adept at interpreting graphical data and generating structured JSON configurations for Plotly"""

    question_prompt = prompt

    image_prompt = input_image_setup(image_loc)
    prompt_parts = [input_prompt, image_prompt[0], question_prompt]
    response = model.generate_content(prompt_parts)
    return str(response.text)

In [6]:
# Code to get information using Gemini model

def gemini_model(prompt):
    
    genai.configure(api_key=gemini_key)

    model = genai.GenerativeModel(model_name="gemini-1.5-pro")

    template = prompt

    response = model.generate_content(template)
    return str(response.text)

In [4]:
info = get_image_info("newplot.png", "extract data")

In [7]:
print(info)
gemini_model(info+"what is this yaar?") 

```json
{
  "data": [
    {
      "x": [
        "2010",
        "2011",
        "2012",
        "2013",
        "2014",
        "2015",
        "2016",
        "2017",
        "2018",
        "2019",
        "2020"
      ],
      "y": [
        "12288",
        "15697",
        "15713",
        "15534",
        "15433",
        "15648",
        "15558",
        "16217",
        "15060",
        "14464",
        "13663"
      ],
      "type": "bar"
    }
  ],
  "layout": {
    "title": {
      "text": "Revenue in Million Euros",
      "font": {
        "size": 24
      }
    },
    "xaxis": {
      "title": {
        "text": "Years",
        "font": {
          "color": "darkslateblue"
        }
      }
    },
    "yaxis": {
      "title": {
        "text": "Revenue (Millions)",
        "font": {
          "color": "darkslateblue"
        }
      }
    }
  }
}
```


'This is a JSON (JavaScript Object Notation) representation of data meant to be plotted as a bar chart. Let\'s break it down:\n\n**1. Data Structure:**\n\n* **`data`**: This is the core of your chart. It\'s an array, currently containing only one object. This object represents a single bar chart dataset. If you had multiple datasets to compare, you\'d have multiple objects within this array.\n* **`x`**: Holds the values for the x-axis (horizontal), which are the years from 2010 to 2020 in your case.\n* **`y`**: Contains the corresponding values for the y-axis (vertical). These are likely revenue figures in million euros, matching each year.\n* **`type`**:  Specifies the chart type, which is set to "bar." \n\n**2. Layout and Styling:**\n\n* **`layout`**:  Defines the overall appearance of your chart.\n* **`title`**:  Sets the title of your chart.\n    * **`text`**: The actual title text.\n    * **`font`**:  Styling for the title text (e.g., size 24).\n* **`xaxis`**:  Configures the x-ax

In [4]:
import time
import json
import os
import csv
from docx import Document

def trail_run_v2(image_loc, prompt):

    # Extract base name of the image without extension
    image_name = os.path.splitext(os.path.basename(image_loc))[0]
    # Create the target directory based on the image name
    json_folder = f'D:/Chart/waterfall_charts/json/{image_name}'    # Extracted JSON directory
    os.makedirs(json_folder, exist_ok=True)
    
    # Extracted CSV directory
    csv_folder = 'D:/Chart/waterfall_charts/extracted_tables'   # Extracted CSV directory
    os.makedirs(csv_folder, exist_ok=True)
    extracted_csv_path = os.path.join(csv_folder, f'{image_name}.csv')
    
    # Placeholder functions for simulation

    extracted_json = get_image_info(image_loc, prompt)
    print("Extracted Json:\n", extracted_json)
    
    data_json = extract_json_section(extracted_json, "data")
    layout_json = extract_json_section(extracted_json, "layout")
    config_json = extract_json_section(extracted_json, "config")
    csv_data = extract_json_section(extracted_json, "csv")
    
    # Convert JSON strings to Python dictionaries
    data = json.loads(data_json) if data_json else None
    layout = json.loads(layout_json) if layout_json else None
    config = json.loads(config_json) if config_json else None
    
    # Save extracted JSON files
    with open(os.path.join(json_folder, 'data.json'), 'w') as f:
        json.dump(data, f, indent=4)
    with open(os.path.join(json_folder, 'layout.json'), 'w') as f:
        json.dump(layout, f, indent=4)
    with open(os.path.join(json_folder, 'config.json'), 'w') as f:
        json.dump(config, f, indent=4)
    
    # Save extracted CSV data
    if csv_data:
        with open(extracted_csv_path, 'w', newline='') as f:
            csv_writer = csv.writer(f)
            csv_reader = csv.reader(csv_data.splitlines())
            csv_writer.writerows(csv_reader)
    
    

In [3]:
# Code to get information using Gemini model

def gemini_model(prompt):
    
    genai.configure(api_key=gemini_key)

    model = genai.GenerativeModel(model_name="gemini-pro")

    template = prompt

    response = model.generate_content(template)
    return str(response.text)

In [5]:
import os
import glob

def process_all_images(png_folder, prompt):
    # Ensure the "Hey Cortana" folder exists
    csv_folder = "charts/waterfall_charts/prompt3"
    os.makedirs(csv_folder, exist_ok=True)
    
    # Find all PNG files in the png_folder
    png_files = glob.glob(os.path.join(png_folder, '*.png'))
    for png_file in png_files:
        try:
            info = get_image_info(png_file, prompt)
            print(info)
            csv_example = '''  
            Category,Value
            Beginning,0
            Planned 1,600
            Planned 2,300
            Actual 1,-450
            Actual 2,-200
            Net Allocation,250

            '''
            prompt2 = f'''from the given code {info} generate csv of datapoints plotted within <csv> </csv> 
            example csv format {csv_example} tags. Make sure you give correctly formatted csv data'''
            csv_data = gemini_model(prompt2)
            print(csv_data)
            csv2 = extract_json_section(csv_data, 'csv')
            print(csv2)
            
            # Save the CSV data with the same filename as the image file, but with .csv extension
            base_name = os.path.basename(png_file)
            csv_filename = os.path.splitext(base_name)[0] + '.csv'
            csv_path = os.path.join(csv_folder, csv_filename)
            with open(csv_path, 'w') as csv_file:
                csv_file.write(csv2)
                
        except Exception as e:
            with open('error.log', 'a') as f:
                f.write(f"Error processing {png_file}: {e}\n")
            print(f"Error processing {png_file}: {e}")
            

with open('prompts/waterfall/prompt3.txt', 'r') as file:
    prompt = file.read()
# Example usage
png_folder = 'charts/waterfall_charts/img'


process_all_images(png_folder, prompt)

  ### Graph Analysis:

1. **Extract Data Points** for Waterfall Chart:

 - **x** (categories):
   - Beginning
   - Sales 1
   - Returns 1
   - Net Sales 1
   - Expenses 1
   - Expenses 2
   - Profit 1
   - Profit 2
   - Final Profit
 - **y** (values):
   - 500
   - 200
   - -50
   - 700
   - -100
   - -120
   - 150
   - 100
   - 400
 - **Changes**:
   - Increases:
     - Sales 1
     - Net Sales 1
     - Profit 1
     - Profit 2
     - Final Profit
   - Decreases:
     - Returns 1
     - Expenses 1
     - Expenses 2

#### Generate Python Code to Regenerate Chart:

1. **Python Code (`<code>...</code>`)**:

```python
import plotly.graph_objects as go

# Define data for the waterfall chart
x_categories = ["Beginning", "Sales 1", "Returns 1", "Net Sales 1", "Expenses 1", "Expenses 2", "Profit 1", "Profit 2", "Final Profit"]
y_values = [500, 200, -50, 700, -100, -120, 150, 100, 400]
measure = ["absolute", "relative", "relative", "absolute", "relative", "relative", "absolute", "relative", "a

KeyboardInterrupt: 

In [4]:
with open('prompts/bar/prompt3.txt', 'r') as file:
    prompt = file.read()

In [5]:
info = get_image_info('charts/bar/img/two_col_104895.png', prompt)

In [6]:
print(info)

 ### Bar Chart Data Extraction:

1. **Data Points**:
   - `x` (Categories):
     - 2010, 2011, 2012, 2013, 2014, 2015, 2016, 2017, 2018, 2019, 2020
   - `y` (Values):
     - 12288, 15697, 15713, 15534, 15433, 15648, 15558, 16217, 15060, 14464, 13663
   - No additional information is provided in the image.

### Python Code Generation:

1. **Python Code (`<code>...</code>`)**:
   ```python
import plotly.graph_objects as go

# Define data for the bar chart
x_categories = ['2010', '2011', '2012', '2013', '2014', '2015', '2016', '2017', '2018', '2019', '2020']
y_values = [12288, 15697, 15713, 15534, 15433, 15648, 15558, 16217, 15060, 14464, 13663]

# Create trace for the bar chart
fig = go.Figure(go.Bar(
    x=x_categories,
    y=y_values,
    marker_color='rgb(55, 83, 109)',  # Customize color as needed
    text=y_values,
    textposition='auto'  # Adjust text position as needed
))

# Customize layout
fig.update_layout(
    title='Revenue in Million Euros',
    title_font_family='Arial, sa

In [11]:
import plotly.graph_objects as go

# Define data for the bar chart
x_categories = ['2010', '2011', '2012', '2013', '2014', '2015', '2016', '2017', '2018', '2019', '2020']
y_values = [12288, 15697, 15713, 15534, 15433, 15648, 15558, 16217, 15060, 14464, 13663]

# Create trace for the bar chart
fig = go.Figure(go.Bar(
    x=x_categories,
    y=y_values,
    marker_color='rgb(55, 83, 109)',  # Customize color as needed
    text=y_values,
    textposition='auto'  # Adjust text position as needed
))

# Customize layout
fig.update_layout(
    title='Revenue in Million Euros',
    title_font_family='Arial, sans-serif',
    title_font_size=24,
    title_font_color='#000000',
    xaxis_title='Years',
    xaxis_title_font=dict(
        family='Arial, sans-serif',
        size=18,
        color='#000000'
    ),
    yaxis_title='Revenue (Millions)',
    yaxis_title_font=dict(
        family='Arial, sans-serif',
        size=18,
        color='#000000'
    ),
    plot_bgcolor='white'
)

# Display the chart
fig.show()

In [17]:
editing_info = f'Extract the data from 2015 to 2020 and writre replot code for  {info} . the laguage of plotting code is in python in plotly'
new_info = gemini_model(editing_info)
print(new_info)

### Data Extraction (2015-2020):

| Year | Revenue (Millions) |
|---|---|
| 2015 | 15648 |
| 2016 | 15558 |
| 2017 | 16217 |
| 2018 | 15060 |
| 2019 | 14464 |
| 2020 | 13663 |

### Python Code Replot (for Extracted Data):

```python
import plotly.graph_objects as go

# Define data for the bar chart
x_categories = ['2015', '2016', '2017', '2018', '2019', '2020']
y_values = [15648, 15558, 16217, 15060, 14464, 13663]

# Create trace for the bar chart
fig = go.Figure(go.Bar(
    x=x_categories,
    y=y_values,
    marker_color='rgb(55, 83, 109)',  # Customize color as needed
    text=y_values,
    textposition='auto'  # Adjust text position as needed
))

# Customize layout
fig.update_layout(
    title='Revenue in Million Euros (2015-2020)',
    title_font_family='Arial, sans-serif',
    title_font_size=24,
    title_font_color='#000000',
    xaxis_title='Years',
    xaxis_title_font=dict(
        family='Arial, sans-serif',
        size=18,
        color='#000000'
    ),
    yaxis_title='R

In [16]:
import plotly.graph_objects as go

# बार चार्ट के लिए डेटा को परिभाषित करें
x_categories = ['2010', '2011', '2012', '2013', '2014', '2015', '2016', '2017', '2018', '2019', '2020']
y_values = [12288, 15697, 15713, 15534, 15433, 15648, 15558, 16217, 15060, 14464, 13663]

# बार चार्ट के लिए ट्रेस बनाएँ
fig = go.Figure(go.Bar(
    x=x_categories,
    y=y_values,
    marker_color='rgb(55, 83, 109)',  # आवश्यकतानुसार रंग को अनुकूलित करें
    text=y_values,
    textposition='auto'  # आवश्यकतानुसार टेक्स्ट की स्थिति समायोजित करें
))

# लेआउट को अनुकूलित करें
fig.update_layout(
    title='मिलियन यूरो में रेवेन्यू',
    title_font_family='Arial, sans-serif',
    title_font_size=24,
    title_font_color='#000000',
    xaxis_title='वर्ष',
    xaxis_title_font=dict(
        family='Arial, sans-serif',
        size=18,
        color='#000000'
    ),
    yaxis_title='रेवेन्यू (मिलियन)',
    yaxis_title_font=dict(
        family='Arial, sans-serif',
        size=18,
        color='#000000'
    ),
    plot_bgcolor='white'
)

# चार्ट दिखाएँ
fig.show()

In [18]:
import plotly.graph_objects as go

# Define data for the bar chart
x_categories = ['2015', '2016', '2017', '2018', '2019', '2020']
y_values = [15648, 15558, 16217, 15060, 14464, 13663]

# Create trace for the bar chart
fig = go.Figure(go.Bar(
    x=x_categories,
    y=y_values,
    marker_color='rgb(55, 83, 109)',  # Customize color as needed
    text=y_values,
    textposition='auto'  # Adjust text position as needed
))

# Customize layout
fig.update_layout(
    title='Revenue in Million Euros (2015-2020)',
    title_font_family='Arial, sans-serif',
    title_font_size=24,
    title_font_color='#000000',
    xaxis_title='Years',
    xaxis_title_font=dict(
        family='Arial, sans-serif',
        size=18,
        color='#000000'
    ),
    yaxis_title='Revenue (Millions)',
    yaxis_title_font=dict(
        family='Arial, sans-serif',
        size=18,
        color='#000000'
    ),
    plot_bgcolor='white'
)

# Display the chart
fig.show()

In [19]:
editing_info = f'Change the language of labels to Bhojpuri  {info} . the laguage of plotting code is in python in plotly'
new_info = gemini_model(editing_info)
print(new_info)

### बार चार्ट डेटा एक्सट्रैक्शन:

1. **डेटा पॉइंट्स**:
   - `x` (कैटेगरीज):
     - 2010, 2011, 2012, 2013, 2014, 2015, 2016, 2017, 2018, 2019, 2020
   - `y` (वैल्यूज):
     - 12288, 15697, 15713, 15534, 15433, 15648, 15558, 16217, 15060, 14464, 13663
   - इमेज में कौनों अउरी जानकारी ना दिहल गइल बा।

### Python कोड जनरेशन:

1. **पाइथॉन कोड (`<code>...</code>`)**:
   ```python
import plotly.graph_objects as go

# बार चार्ट के डेटा के परिभाषा करीं
x_categories = ['2010', '2011', '2012', '2013', '2014', '2015', '2016', '2017', '2018', '2019', '2020']
y_values = [12288, 15697, 15713, 15534, 15433, 15648, 15558, 16217, 15060, 14464, 13663]

# बार चार्ट के ट्रेस बनाईं
fig = go.Figure(go.Bar(
    x=x_categories,
    y=y_values,
    marker_color='rgb(55, 83, 109)',  # आपन जरूरत के अनुसार रंग बदल सकेनी
    text=y_values,
    textposition='auto'  # आपन जरूरत के अनुसार टेक्स्ट पोजीशन एडजस्ट करे सकनी
))

# लेआउट कस्टमाइज करीं
fig.update_layout(
    title='मिलियन यूरो में रेवेन्यु',
    title_font_f

In [20]:
import plotly.graph_objects as go

# बार चार्ट के डेटा के परिभाषा करीं
x_categories = ['2010', '2011', '2012', '2013', '2014', '2015', '2016', '2017', '2018', '2019', '2020']
y_values = [12288, 15697, 15713, 15534, 15433, 15648, 15558, 16217, 15060, 14464, 13663]

# बार चार्ट के ट्रेस बनाईं
fig = go.Figure(go.Bar(
    x=x_categories,
    y=y_values,
    marker_color='rgb(55, 83, 109)',  # आपन जरूरत के अनुसार रंग बदल सकेनी
    text=y_values,
    textposition='auto'  # आपन जरूरत के अनुसार टेक्स्ट पोजीशन एडजस्ट करे सकनी
))

# लेआउट कस्टमाइज करीं
fig.update_layout(
    title='मिलियन यूरो में रेवेन्यु',
    title_font_family='Arial, sans-serif',
    title_font_size=24,
    title_font_color='#000000',
    xaxis_title='साल',
    xaxis_title_font=dict(
        family='Arial, sans-serif',
        size=18,
        color='#000000'
    ),
    yaxis_title='रेवेन्यु (मिलियन)',
    yaxis_title_font=dict(
        family='Arial, sans-serif',
        size=18,
        color='#000000'
    ),
    plot_bgcolor='white'
)

# चार्ट डिस्प्ले करीं
fig.show()

In [21]:
editing_info = f'Change the color of bars to light green  {info} . the laguage of plotting code is in python in plotly'
new_info = gemini_model(editing_info)
print(new_info)

1. **Python Code (`<code>...</code>`)**:
   ```python
import plotly.graph_objects as go

# Define data for the bar chart
x_categories = ['2010', '2011', '2012', '2013', '2014', '2015', '2016', '2017', '2018', '2019', '2020']
y_values = [12288, 15697, 15713, 15534, 15433, 15648, 15558, 16217, 15060, 14464, 13663]

# Create trace for the bar chart
fig = go.Figure(go.Bar(
    x=x_categories,
    y=y_values,
    marker_color='lightgreen',  # Change color to light green
    text=y_values,
    textposition='auto'  # Adjust text position as needed
))

# Customize layout
fig.update_layout(
    title='Revenue in Million Euros',
    title_font_family='Arial, sans-serif',
    title_font_size=24,
    title_font_color='#000000',
    xaxis_title='Years',
    xaxis_title_font=dict(
        family='Arial, sans-serif',
        size=18,
        color='#000000'
    ),
    yaxis_title='Revenue (Millions)',
    yaxis_title_font=dict(
        family='Arial, sans-serif',
        size=18,
        color='#0000

In [22]:
import plotly.graph_objects as go

# Define data for the bar chart
x_categories = ['2010', '2011', '2012', '2013', '2014', '2015', '2016', '2017', '2018', '2019', '2020']
y_values = [12288, 15697, 15713, 15534, 15433, 15648, 15558, 16217, 15060, 14464, 13663]

# Create trace for the bar chart
fig = go.Figure(go.Bar(
    x=x_categories,
    y=y_values,
    marker_color='lightgreen',  # Change color to light green
    text=y_values,
    textposition='auto'  # Adjust text position as needed
))

# Customize layout
fig.update_layout(
    title='Revenue in Million Euros',
    title_font_family='Arial, sans-serif',
    title_font_size=24,
    title_font_color='#000000',
    xaxis_title='Years',
    xaxis_title_font=dict(
        family='Arial, sans-serif',
        size=18,
        color='#000000'
    ),
    yaxis_title='Revenue (Millions)',
    yaxis_title_font=dict(
        family='Arial, sans-serif',
        size=18,
        color='#000000'
    ),
    plot_bgcolor='white'
)

# Display the chart
fig.show()

In [25]:
editing_info = f'Convert the given chart to line chart with data points marked. Follow all the guidlines for good data plotting  Chart : {info} . the laguage of plotting code is in python in plotly'
new_info = gemini_model(editing_info)
print(new_info)

### Line Chart with Data Points Marked

 **Python Code (`<code>...</code>`)**:

```python
import plotly.graph_objects as go

# Define data for the line chart
x_categories = ['2010', '2011', '2012', '2013', '2014', '2015', '2016', '2017', '2018', '2019', '2020']
y_values = [12288, 15697, 15713, 15534, 15433, 15648, 15558, 16217, 15060, 14464, 13663]

# Create trace for the line chart
fig = go.Figure(go.Scatter(
    x=x_categories,
    y=y_values,
    mode='lines+markers',  # Specify line and markers
    marker=dict(
        size=8,
        color='rgb(55, 83, 109)',  # Customize color as needed
        symbol='circle'
    ),
    text=y_values,
    textposition='top center'  # Adjust text position as needed
))

# Customize layout
fig.update_layout(
    title='Revenue in Million Euros',
    title_font_family='Arial, sans-serif',
    title_font_size=24,
    title_font_color='#000000',
    xaxis_title='Years',
    xaxis_title_font=dict(
        family='Arial, sans-serif',
        size=18,
  

In [26]:
import plotly.graph_objects as go

# Define data for the line chart
x_categories = ['2010', '2011', '2012', '2013', '2014', '2015', '2016', '2017', '2018', '2019', '2020']
y_values = [12288, 15697, 15713, 15534, 15433, 15648, 15558, 16217, 15060, 14464, 13663]

# Create trace for the line chart
fig = go.Figure(go.Scatter(
    x=x_categories,
    y=y_values,
    mode='lines+markers',  # Specify line and markers
    marker=dict(
        size=8,
        color='rgb(55, 83, 109)',  # Customize color as needed
        symbol='circle'
    ),
    text=y_values,
    textposition='top center'  # Adjust text position as needed
))

# Customize layout
fig.update_layout(
    title='Revenue in Million Euros',
    title_font_family='Arial, sans-serif',
    title_font_size=24,
    title_font_color='#000000',
    xaxis_title='Years',
    xaxis_title_font=dict(
        family='Arial, sans-serif',
        size=18,
        color='#000000'
    ),
    yaxis_title='Revenue (Millions)',
    yaxis_title_font=dict(
        family='Arial, sans-serif',
        size=18,
        color='#000000'
    ),
    plot_bgcolor='white'
)

# Display the chart
fig.show()

In [24]:
import plotly.graph_objects as go

# Define data for the line chart
x_categories = ['2010', '2011', '2012', '2013', '2014', '2015', '2016', '2017', '2018', '2019', '2020']
y_values = [12288, 15697, 15713, 15534, 15433, 15648, 15558, 16217, 15060, 14464, 13663]

# Create trace for the line chart
fig = go.Figure(go.Scatter(
    x=x_categories,
    y=y_values,
    mode='lines',
    line=dict(color='rgb(55, 83, 109)', width=3),
    marker=dict(size=8, color='rgb(55, 83, 109)'),
    text=y_values,
    textposition='top center'
))

# Customize layout
fig.update_layout(
    title='Revenue in Million Euros',
    title_font_family='Arial, sans-serif',
    title_font_size=24,
    title_font_color='#000000',
    xaxis_title='Years',
    xaxis_title_font=dict(
        family='Arial, sans-serif',
        size=18,
        color='#000000'
    ),
    yaxis_title='Revenue (Millions)',
    yaxis_title_font=dict(
        family='Arial, sans-serif',
        size=18,
        color='#000000'
    ),
    plot_bgcolor='white'
)

# Display the chart
fig.show()


1.3.1 Chart Editing
1. Style edits - plot colors, styles (line style, marker style, bar pattern) and font family, font size
- Changing the size of the chart - can be enlarged to visualize that better
- Changes in the font styles and font size(e.g. legend, axis labels, Title etc) by changing font
family
- Changing the colors - sometimes colors of multiple lines are not clearly distinguishable. Color
palette can be changed to make it more visually appealing
2. Format - Plot (line - grouped bar, line to stacked bar, grouped bar to stacked bar) - chart
conversion from one type to other type - dividing one chart into multiple chart e.g area plot
(explain ??)
1
3. layout - legend (position, internal layout), Axes (Grid Lines)
4. Data Centric - Filtering - range wise, series based
- Data addition - add/update data points, data series
- Subset of the chart and enlarging it - helps in focusing on some parts of the chart
5. Handle Missing Data Values
6. Chart Summarization - Providing a header/caption to chart - Giving higher level view of chart
by summarizing smaller points
- Fact Checking - whether given title matches the data summarization or not
Tested on chartQA dataset. Bar, line chart are giving quite good results
for other types of charts, trying chain of thought prompts to find out the type of chart in first step
and reading data points in the second step
Reading of multilingual charts and translation of the same to other languages. create a synthetic
multilingual chart dataset
Explore the possibility of conversion of complex charts.
Different evaluation metrics are given for each task.

In [8]:
editing_info = f'Change the size of chart , enlarge it chart:  {info} . the laguage of plotting code is in python in plotly, give the complete code'
new_info = gemini_model(editing_info)
print(new_info)

```python
import plotly.graph_objects as go

# Define data for the bar chart
x_categories = ['2010', '2011', '2012', '2013', '2014', '2015', '2016', '2017', '2018', '2019', '2020']
y_values = [12288, 15697, 15713, 15534, 15433, 15648, 15558, 16217, 15060, 14464, 13663]

# Create trace for the bar chart
fig = go.Figure(go.Bar(
    x=x_categories,
    y=y_values,
    marker_color='rgb(55, 83, 109)',  # Customize color as needed
    text=y_values,
    textposition='auto'  # Adjust text position as needed
))

# Customize layout
fig.update_layout(
    title='Revenue in Million Euros',
    title_font_family='Arial, sans-serif',
    title_font_size=24,
    title_font_color='#000000',
    xaxis_title='Years',
    xaxis_title_font=dict(
        family='Arial, sans-serif',
        size=18,
        color='#000000'
    ),
    yaxis_title='Revenue (Millions)',
    yaxis_title_font=dict(
        family='Arial, sans-serif',
        size=18,
        color='#000000'
    ),
    plot_bgcolor='white',
   

In [9]:
import plotly.graph_objects as go

# Define data for the bar chart
x_categories = ['2010', '2011', '2012', '2013', '2014', '2015', '2016', '2017', '2018', '2019', '2020']
y_values = [12288, 15697, 15713, 15534, 15433, 15648, 15558, 16217, 15060, 14464, 13663]

# Create trace for the bar chart
fig = go.Figure(go.Bar(
    x=x_categories,
    y=y_values,
    marker_color='rgb(55, 83, 109)',  # Customize color as needed
    text=y_values,
    textposition='auto'  # Adjust text position as needed
))

# Customize layout
fig.update_layout(
    title='Revenue in Million Euros',
    title_font_family='Arial, sans-serif',
    title_font_size=24,
    title_font_color='#000000',
    xaxis_title='Years',
    xaxis_title_font=dict(
        family='Arial, sans-serif',
        size=18,
        color='#000000'
    ),
    yaxis_title='Revenue (Millions)',
    yaxis_title_font=dict(
        family='Arial, sans-serif',
        size=18,
        color='#000000'
    ),
    plot_bgcolor='white',
    width=800,
    height=600
)

# Display the chart
fig.show()

In [10]:
editing_info = f'Change the font family and font stle of chart labels , axes {info} . the laguage of plotting code is in python in plotly'
new_info = gemini_model(editing_info)
print(new_info)

```python
import plotly.graph_objects as go

# Define data for the bar chart
x_categories = ['2010', '2011', '2012', '2013', '2014', '2015', '2016', '2017', '2018', '2019', '2020']
y_values = [12288, 15697, 15713, 15534, 15433, 15648, 15558, 16217, 15060, 14464, 13663]

# Create trace for the bar chart
fig = go.Figure(go.Bar(
    x=x_categories,
    y=y_values,
    marker_color='rgb(55, 83, 109)',  # Customize color as needed
    text=y_values,
    textposition='auto'  # Adjust text position as needed
))

# Customize layout
fig.update_layout(
    title='Revenue in Million Euros',
    title_font_family='Arial, sans-serif',
    title_font_size=24,
    title_font_color='#000000',
    xaxis_title='Years',
    xaxis_title_font=dict(
        family='Arial, sans-serif',
        size=18,
        color='#000000'
    ),
    yaxis_title='Revenue (Millions)',
    yaxis_title_font=dict(
        family='Arial, sans-serif',
        size=18,
        color='#000000'
    ),
    plot_bgcolor='white',
   

In [11]:
import plotly.graph_objects as go

# Define data for the bar chart
x_categories = ['2010', '2011', '2012', '2013', '2014', '2015', '2016', '2017', '2018', '2019', '2020']
y_values = [12288, 15697, 15713, 15534, 15433, 15648, 15558, 16217, 15060, 14464, 13663]

# Create trace for the bar chart
fig = go.Figure(go.Bar(
    x=x_categories,
    y=y_values,
    marker_color='rgb(55, 83, 109)',  # Customize color as needed
    text=y_values,
    textposition='auto'  # Adjust text position as needed
))

# Customize layout
fig.update_layout(
    title='Revenue in Million Euros',
    title_font_family='Arial, sans-serif',
    title_font_size=24,
    title_font_color='#000000',
    xaxis_title='Years',
    xaxis_title_font=dict(
        family='Arial, sans-serif',
        size=18,
        color='#000000'
    ),
    yaxis_title='Revenue (Millions)',
    yaxis_title_font=dict(
        family='Arial, sans-serif',
        size=18,
        color='#000000'
    ),
    plot_bgcolor='white',
    font_family='Arial, sans-serif',  # Sets the font family for all text elements
    font_color='#000000'  # Sets the font color for all text elements
)

# Display the chart
fig.show()

In [12]:
editing_info = f'Add grid lines to the chart  {info} . the laguage of plotting code is in python in plotly'
new_info = gemini_model(editing_info)
print(new_info)

```python
import plotly.graph_objects as go

# Define data for the bar chart
x_categories = ['2010', '2011', '2012', '2013', '2014', '2015', '2016', '2017', '2018', '2019', '2020']
y_values = [12288, 15697, 15713, 15534, 15433, 15648, 15558, 16217, 15060, 14464, 13663]

# Create trace for the bar chart
fig = go.Figure(go.Bar(
    x=x_categories,
    y=y_values,
    marker_color='rgb(55, 83, 109)',  # Customize color as needed
    text=y_values,
    textposition='auto'  # Adjust text position as needed
))

# Add grid lines
fig.update_xaxes(showgrid=True, gridcolor='#D3D3D3')
fig.update_yaxes(showgrid=True, gridcolor='#D3D3D3')

# Customize layout
fig.update_layout(
    title='Revenue in Million Euros',
    title_font_family='Arial, sans-serif',
    title_font_size=24,
    title_font_color='#000000',
    xaxis_title='Years',
    xaxis_title_font=dict(
        family='Arial, sans-serif',
        size=18,
        color='#000000'
    ),
    yaxis_title='Revenue (Millions)',
    yaxis_title_

In [13]:
import plotly.graph_objects as go

# Define data for the bar chart
x_categories = ['2010', '2011', '2012', '2013', '2014', '2015', '2016', '2017', '2018', '2019', '2020']
y_values = [12288, 15697, 15713, 15534, 15433, 15648, 15558, 16217, 15060, 14464, 13663]

# Create trace for the bar chart
fig = go.Figure(go.Bar(
    x=x_categories,
    y=y_values,
    marker_color='rgb(55, 83, 109)',  # Customize color as needed
    text=y_values,
    textposition='auto'  # Adjust text position as needed
))

# Add grid lines
fig.update_xaxes(showgrid=True, gridcolor='#D3D3D3')
fig.update_yaxes(showgrid=True, gridcolor='#D3D3D3')

# Customize layout
fig.update_layout(
    title='Revenue in Million Euros',
    title_font_family='Arial, sans-serif',
    title_font_size=24,
    title_font_color='#000000',
    xaxis_title='Years',
    xaxis_title_font=dict(
        family='Arial, sans-serif',
        size=18,
        color='#000000'
    ),
    yaxis_title='Revenue (Millions)',
    yaxis_title_font=dict(
        family='Arial, sans-serif',
        size=18,
        color='#000000'
    ),
    plot_bgcolor='white'
)

# Display the chart
fig.show()

In [15]:
editing_info = f'summarize the chart  {info} '
new_info = gemini_model(editing_info)
print(new_info)

**Bar Chart Data Summary**

**Data:**

* **Years (Categories, x-axis):** 2010-2020
* **Revenue (Values, y-axis):** 12288-13663 (in million euros)

**Python Code Snippet:**

```python
import plotly.graph_objects as go

# Data
x_categories = ['2010', '2011', '2012', '2013', '2014', '2015', '2016', '2017', '2018', '2019', '2020']
y_values = [12288, 15697, 15713, 15534, 15433, 15648, 15558, 16217, 15060, 14464, 13663]

# Create Bar Chart
fig = go.Figure(go.Bar(x=x_categories, y=y_values, marker_color='rgb(55, 83, 109)', text=y_values, textposition='auto'))

# Customize Layout
fig.update_layout(title='Revenue in Million Euros', xaxis_title='Years', yaxis_title='Revenue (Millions)', plot_bgcolor='white')

# Display Chart
fig.show()
```


In [16]:
import plotly.graph_objects as go

# Data
x_categories = ['2010', '2011', '2012', '2013', '2014', '2015', '2016', '2017', '2018', '2019', '2020']
y_values = [12288, 15697, 15713, 15534, 15433, 15648, 15558, 16217, 15060, 14464, 13663]

# Create Bar Chart
fig = go.Figure(go.Bar(x=x_categories, y=y_values, marker_color='rgb(55, 83, 109)', text=y_values, textposition='auto'))

# Customize Layout
fig.update_layout(title='Revenue in Million Euros', xaxis_title='Years', yaxis_title='Revenue (Millions)', plot_bgcolor='white')

# Display Chart
fig.show()

In [17]:
editing_info = f'Add a header and a caption to the chart  {info} . the laguage of plotting code is in python in plotly'
new_info = gemini_model(editing_info)
print(new_info)

### Bar Chart Data Extraction:

**Header**: Revenue over the Years

**Caption**: This chart shows the revenue (in million euros) of a company from 2010 to 2020.

1. **Data Points**:
   - `x` (Categories):
     - 2010, 2011, 2012, 2013, 2014, 2015, 2016, 2017, 2018, 2019, 2020
   - `y` (Values):
     - 12288, 15697, 15713, 15534, 15433, 15648, 15558, 16217, 15060, 14464, 13663
   - No additional information is provided in the image.

### Python Code Generation:

1. **Python Code (`<code>...</code>`)**:
   ```python
import plotly.graph_objects as go

# Define data for the bar chart
x_categories = ['2010', '2011', '2012', '2013', '2014', '2015', '2016', '2017', '2018', '2019', '2020']
y_values = [12288, 15697, 15713, 15534, 15433, 15648, 15558, 16217, 15060, 14464, 13663]

# Create trace for the bar chart
fig = go.Figure(go.Bar(
    x=x_categories,
    y=y_values,
    marker_color='rgb(55, 83, 109)',  # Customize color as needed
    text=y_values,
    textposition='auto'  # Adjust text po

In [18]:
import plotly.graph_objects as go

# Define data for the bar chart
x_categories = ['2010', '2011', '2012', '2013', '2014', '2015', '2016', '2017', '2018', '2019', '2020']
y_values = [12288, 15697, 15713, 15534, 15433, 15648, 15558, 16217, 15060, 14464, 13663]

# Create trace for the bar chart
fig = go.Figure(go.Bar(
    x=x_categories,
    y=y_values,
    marker_color='rgb(55, 83, 109)',  # Customize color as needed
    text=y_values,
    textposition='auto'  # Adjust text position as needed
))

# Customize layout
fig.update_layout(
    title='Revenue over the Years',
    title_font_family='Arial, sans-serif',
    title_font_size=24,
    title_font_color='#000000',
    xaxis_title='Years',
    xaxis_title_font=dict(
        family='Arial, sans-serif',
        size=18,
        color='#000000'
    ),
    yaxis_title='Revenue (Millions)',
    yaxis_title_font=dict(
        family='Arial, sans-serif',
        size=18,
        color='#000000'
    ),
    plot_bgcolor='white'
)

# Display the chart
fig.show()

In [19]:
editing_info = f'following the trend add data points for year 2021 and 2022 , 2023, 2024 {info} . the laguage of plotting code is in python in plotly'
new_info = gemini_model(editing_info)
print(new_info)

1. **Data Points**:
   - `x` (Categories):
     - 2010, 2011, 2012, 2013, 2014, 2015, 2016, 2017, 2018, 2019, 2020, 2021, 2022, 2023, 2024
   - `y` (Values):
     - 12288, 15697, 15713, 15534, 15433, 15648, 15558, 16217, 15060, 14464, 13663, 13345, 13028, 12710, 12393

### Python Code Generation:

1. **Python Code (`<code>...</code>`)**:
   ```python
# Import necessary libraries
import plotly.graph_objects as go

# Define data for the bar chart
x_categories = ['2010', '2011', '2012', '2013', '2014', '2015', '2016', '2017', '2018', '2019', '2020', '2021', '2022', '2023', '2024']
y_values = [12288, 15697, 15713, 15534, 15433, 15648, 15558, 16217, 15060, 14464, 13663, 13345, 13028, 12710, 12393]

# Create trace for the bar chart
fig = go.Figure(go.Bar(
    x=x_categories,
    y=y_values,
    marker_color='rgb(55, 83, 109)',  # Customize color as needed
    text=y_values,
    textposition='auto'  # Adjust text position as needed
))

# Customize layout
fig.update_layout(
    title='Revenue 

In [20]:
import plotly.graph_objects as go

# Define data for the bar chart
x_categories = ['2010', '2011', '2012', '2013', '2014', '2015', '2016', '2017', '2018', '2019', '2020', '2021', '2022', '2023', '2024']
y_values = [12288, 15697, 15713, 15534, 15433, 15648, 15558, 16217, 15060, 14464, 13663, 13345, 13028, 12710, 12393]

# Create trace for the bar chart
fig = go.Figure(go.Bar(
    x=x_categories,
    y=y_values,
    marker_color='rgb(55, 83, 109)',  # Customize color as needed
    text=y_values,
    textposition='auto'  # Adjust text position as needed
))

# Customize layout
fig.update_layout(
    title='Revenue in Million Euros',
    title_font_family='Arial, sans-serif',
    title_font_size=24,
    title_font_color='#000000',
    xaxis_title='Years',
    xaxis_title_font=dict(
        family='Arial, sans-serif',
        size=18,
        color='#000000'
    ),
    yaxis_title='Revenue (Millions)',
    yaxis_title_font=dict(
        family='Arial, sans-serif',
        size=18,
        color='#000000'
    ),
    plot_bgcolor='white'
)

# Display the chart
fig.show()

In [25]:
editing_info = f'extract only those years where value is greater than 15000 from  {info} and give code to plot those years. the laguage of plotting code is in python in plotly'
new_info = gemini_model(editing_info)
print(new_info)

```python
import plotly.graph_objects as go

# Define data for the bar chart
x_categories = ['2010', '2011', '2012', '2013', '2014', '2015', '2016', '2017', '2018', '2019', '2020']
y_values = [12288, 15697, 15713, 15534, 15433, 15648, 15558, 16217, 15060, 14464, 13663]

# Extract years where value is greater than 15000
filtered_years = [year for year, value in zip(x_categories, y_values) if value > 15000]

# Create trace for the bar chart
fig = go.Figure(go.Bar(
    x=filtered_years,
    y=[value for value in y_values if value > 15000],
    marker_color='rgb(55, 83, 109)',  # Customize color as needed
    text=[value for value in y_values if value > 15000],
    textposition='auto'  # Adjust text position as needed
))

# Customize layout
fig.update_layout(
    title='Years with Revenue > 15000 Euros',
    title_font_family='Arial, sans-serif',
    title_font_size=24,
    title_font_color='#000000',
    xaxis_title='Years',
    xaxis_title_font=dict(
        family='Arial, sans-serif',
 

In [26]:
import plotly.graph_objects as go

# Define data for the bar chart
x_categories = ['2010', '2011', '2012', '2013', '2014', '2015', '2016', '2017', '2018', '2019', '2020']
y_values = [12288, 15697, 15713, 15534, 15433, 15648, 15558, 16217, 15060, 14464, 13663]

# Extract years where value is greater than 15000
filtered_years = [year for year, value in zip(x_categories, y_values) if value > 15000]

# Create trace for the bar chart
fig = go.Figure(go.Bar(
    x=filtered_years,
    y=[value for value in y_values if value > 15000],
    marker_color='rgb(55, 83, 109)',  # Customize color as needed
    text=[value for value in y_values if value > 15000],
    textposition='auto'  # Adjust text position as needed
))

# Customize layout
fig.update_layout(
    title='Years with Revenue > 15000 Euros',
    title_font_family='Arial, sans-serif',
    title_font_size=24,
    title_font_color='#000000',
    xaxis_title='Years',
    xaxis_title_font=dict(
        family='Arial, sans-serif',
        size=18,
        color='#000000'
    ),
    yaxis_title='Revenue (Millions)',
    yaxis_title_font=dict(
        family='Arial, sans-serif',
        size=18,
        color='#000000'
    ),
    plot_bgcolor='white'
)

# Display the chart
fig.show()


In [27]:
editing_info = f'Change this to horizontal chart  {info} . the laguage of plotting code is in python in plotly'
new_info = gemini_model(editing_info)
print(new_info)

### Horizontal Bar Chart Data Extraction:

1. **Data Points**:
   - `x` (Categories):
     - 12288, 15697, 15713, 15534, 15433, 15648, 15558, 16217, 15060, 14464, 13663
   - `y` (Values):
     - 2010, 2011, 2012, 2013, 2014, 2015, 2016, 2017, 2018, 2019, 2020
   - No additional information is provided in the image.

### Python Code Generation:

1. **Python Code (`<code>...</code>`)**:
   ```python
import plotly.graph_objects as go

# Define data for the bar chart
x_values = [12288, 15697, 15713, 15534, 15433, 15648, 15558, 16217, 15060, 14464, 13663]
y_categories = ['2010', '2011', '2012', '2013', '2014', '2015', '2016', '2017', '2018', '2019', '2020']

# Create trace for the bar chart
fig = go.Figure(go.Bar(
    x=x_values,
    y=y_categories,
    orientation='h',  # Set orientation to horizontal
    marker_color='rgb(55, 83, 109)',  # Customize color as needed
    text=x_values,
    textposition='auto'  # Adjust text position as needed
))

# Customize layout
fig.update_layout(
    ti

In [28]:
import plotly.graph_objects as go

# Define data for the bar chart
x_values = [12288, 15697, 15713, 15534, 15433, 15648, 15558, 16217, 15060, 14464, 13663]
y_categories = ['2010', '2011', '2012', '2013', '2014', '2015', '2016', '2017', '2018', '2019', '2020']

# Create trace for the bar chart
fig = go.Figure(go.Bar(
    x=x_values,
    y=y_categories,
    orientation='h',  # Set orientation to horizontal
    marker_color='rgb(55, 83, 109)',  # Customize color as needed
    text=x_values,
    textposition='auto'  # Adjust text position as needed
))

# Customize layout
fig.update_layout(
    title='Revenue in Million Euros',
    title_font_family='Arial, sans-serif',
    title_font_size=24,
    title_font_color='#000000',
    xaxis_title='Revenue (Millions)',
    xaxis_title_font=dict(
        family='Arial, sans-serif',
        size=18,
        color='#000000'
    ),
    yaxis_title='Years',
    yaxis_title_font=dict(
        family='Arial, sans-serif',
        size=18,
        color='#000000'
    ),
    plot_bgcolor='white'
)

# Display the chart
fig.show()

In [30]:
editing_info = f'Change the language of labels to Bengali {info} . the laguage of plotting code and the comments are in python in plotly'
new_info = gemini_model(editing_info)
print(new_info)

### বার চার্ট তথ্য নিষ্কাশন:

1. **ডেটা পয়েন্ট:**
   - `x` (বিভাগ):
     - ২০১০, ২০১১, ২০১২, ২০১৩, ২০১৪, ২০১৫, ২০১৬, ২০১৭, ২০১৮, ২০১৯, ২০২০
   - `y` (মান):
     - ১২২৮৮, ১৫৬৯৭, ১৫৭১৩, ১৫৫৩৪, ১৫৪৩৩, ১৫৬৪৮, ১৫৫৫৮, ১৬২১৭, ১৫০৬০, ১৪৪৬৪, ১৩৬৬৩
   - চিত্রে কোন অতিরিক্ত তথ্য দেওয়া নেই।

### পাইথন কোড জেনারেশন:

1. **পাইথন কোড (`<code>...</code>`)**:
   ```python
import plotly.graph_objects as go

# বার চার্টের জন্য ডেটা নির্ধার করুন
x_categories = ['২০১০', '২০১১', '২০১২', '২০১৩', '২০১৪', '২০১৫', '২০১৬', '২০১৭', '২০১৮', '২০১৯', '২০২০']
y_values = [১২২৮৮, ১৫৬৯৭, ১৫৭১৩, ১৫৫৩৪, ১৫৪৩৩, ১৫৬৪৮, ১৫৫৫৮, ১৬২১৭, ১৫০৬০, ১৪৪৬৪, ১৩৬৬৩]

# বার চার্টের জন্য ট্রেস তৈরি করুন
fig = go.Figure(go.Bar(
    x=x_categories,
    y=y_values,
    marker_color='rgb(55, 83, 109)',  # আপনার প্রয়োজন অনুযায়ী রঙ কাস্টমাইজ করুন
    text=y_values,
    textposition='auto'  # আপনার প্রয়োজন অনুযায়ী টেক্সটের অবস্থান কাস্টমাইজ করুন
))

# লেআউট কাস্টমাইজ করুন
fig.update_layout(
    title='কোটি ইউরোয় রেভিনিউ',
    title_font_family='

In [32]:
import plotly.graph_objects as go

# বার চার্টের জন্য ডেটা নির্ধার করুন
x_categories = ['২০১০', '২০১১', '২০১২', '২০১৩', '২০১৪', '২০১৫', '২০১৬', '২০১৭', '২০১৮', '২০১৯', '২০২০']
y_categories = ['2010', '2011', '2012', '2013', '2014', '2015', '2016', '2017', '2018', '2019', '2020']

# বার চার্টের জন্য ট্রেস তৈরি করুন
fig = go.Figure(go.Bar(
    x=x_categories,
    y=y_values,
    marker_color='rgb(55, 83, 109)',  # আপনার প্রয়োজন অনুযায়ী রঙ কাস্টমাইজ করুন
    text=y_values,
    textposition='auto'  # আপনার প্রয়োজন অনুযায়ী টেক্সটের অবস্থান কাস্টমাইজ করুন
))

# লেআউট কাস্টমাইজ করুন
fig.update_layout(
    title='কোটি ইউরোয় রেভিনিউ',
    title_font_family='Arial, sans-serif',
    title_font_size=24,
    title_font_color='#000000',
    xaxis_title='বছর',
    xaxis_title_font=dict(
        family='Arial, sans-serif',
        size=18,
        color='#000000'
    ),
    yaxis_title='রেভিনিউ (কোটি)',
    yaxis_title_font=dict(
        family='Arial, sans-serif',
        size=18,
        color='#000000'
    ),
    plot_bgcolor='white'
)

# চার্টটি প্রদর্শন করুন
fig.show()

In [34]:
editing_info = f'Change the axes title fonts  to roboto and font color to blue of the chart   {info} . the laguage of plotting code is in python in plotly'
new_info = gemini_model(editing_info)
print(new_info)

```python
import plotly.graph_objects as go

# Define data for the bar chart
x_categories = ['2010', '2011', '2012', '2013', '2014', '2015', '2016', '2017', '2018', '2019', '2020']
y_values = [12288, 15697, 15713, 15534, 15433, 15648, 15558, 16217, 15060, 14464, 13663]

# Create trace for the bar chart
fig = go.Figure(go.Bar(
    x=x_categories,
    y=y_values,
    marker_color='rgb(55, 83, 109)',  # Customize color as needed
    text=y_values,
    textposition='auto'  # Adjust text position as needed
))

# Customize layout
fig.update_layout(
    title='Revenue in Million Euros',
    title_font_family='Arial, sans-serif',
    title_font_size=24,
    title_font_color='#000000',
    xaxis_title='Years',
    xaxis_title_font=dict(
        family='Roboto',
        size=18,
        color='blue'  # Change the font color to blue
    ),
    yaxis_title='Revenue (Millions)',
    yaxis_title_font=dict(
        family='Roboto',
        size=18,
        color='blue'  # Change the font color to blu

In [35]:
import plotly.graph_objects as go

# Define data for the bar chart
x_categories = ['2010', '2011', '2012', '2013', '2014', '2015', '2016', '2017', '2018', '2019', '2020']
y_values = [12288, 15697, 15713, 15534, 15433, 15648, 15558, 16217, 15060, 14464, 13663]

# Create trace for the bar chart
fig = go.Figure(go.Bar(
    x=x_categories,
    y=y_values,
    marker_color='rgb(55, 83, 109)',  # Customize color as needed
    text=y_values,
    textposition='auto'  # Adjust text position as needed
))

# Customize layout
fig.update_layout(
    title='Revenue in Million Euros',
    title_font_family='Arial, sans-serif',
    title_font_size=24,
    title_font_color='#000000',
    xaxis_title='Years',
    xaxis_title_font=dict(
        family='Roboto',
        size=18,
        color='blue'  # Change the font color to blue
    ),
    yaxis_title='Revenue (Millions)',
    yaxis_title_font=dict(
        family='Roboto',
        size=18,
        color='blue'  # Change the font color to blue
    ),
    plot_bgcolor='white'
)

# Display the chart
fig.show()